In [5]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder

In [6]:
# list of transactions from a store.
file = "data/parsed_receipts.pkl"
df = pd.read_pickle(file)

In [7]:
# we need to focus only on some EANs
high_risk_eans = pd.read_pickle("data/high_risk_eans.pkl")

In [8]:
# Data pre-processing
sales_products_df = df[(df['is_discount'] == False)]
sales_products_df = sales_products_df[(sales_products_df['product'] != "Ref.nummer")]
sales_products_df = sales_products_df[(sales_products_df['product'] != "EL-tjänstinformation")]
sales_products_df = sales_products_df[(sales_products_df['subtotal'] > 0)]
sales_products_df = sales_products_df[(sales_products_df['quantity'] > 0)]
sales_products_df.dropna()
sales_products_df['product'].str.strip()

unique_sales_products = sales_products_df['ean'].unique()
num_products = len(unique_sales_products)

sales_products_df = sales_products_df[sales_products_df['ean'].str.isnumeric()]

unique_sales_products = sales_products_df['ean'].unique()

num_products = len(unique_sales_products)


16638
16638


In [9]:
sales_products_df_subset = sales_products_df[sales_products_df['ean'].isin(high_risk_eans)]

In [4]:
# Unique transactions
unique_transactions = sales_products_df['transaction_id'].unique()

num_transactions = len(unique_transactions)

192547


In [5]:
s_products_df = sales_products_df[["transaction_id", "ean", "product"]]

In [6]:
s_products_df.to_pickle("s_products_df.pkl")

In [7]:
# for each transaction, return the list of product names.
data = list(s_products_df.groupby("transaction_id").apply(lambda x: list(x.apply(lambda x: x["product"], axis = 1))))
# data

In [11]:
import pickle
pickle.dump(data, open("transaction_product_array.pkl", "wb"))

In [12]:
# convert data into a format understandble for apriori
te = TransactionEncoder()
te_ary = te.fit(data).transform(data)

In [13]:
te_ary = te_ary.astype("int")

In [16]:
dd = pd.DataFrame(te_ary, columns=te.columns_)

In [ ]:
dd.to_pickle("input_to_aprori.pkl")

In [ ]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

# find frequent itemsets
frequent_itemsets = apriori(dd, min_support=0.001, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))

In [28]:
# get association rules.
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=0.7)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Banan Eko),(Avocado),0.123850,0.046965,0.010964,0.088523,1.884858,0.005147,1.045594
1,(Avocado),(Banan Eko),0.046965,0.123850,0.010964,0.233440,1.884858,0.005147,1.142963
2,(Gurka),(Avocado),0.084660,0.046965,0.012184,0.143918,3.064347,0.008208,1.113251
3,(Avocado),(Gurka),0.046965,0.084660,0.012184,0.259427,3.064347,0.008208,1.235989
4,(Avocado),(Miljöpåse),0.046965,0.366217,0.021184,0.451067,1.231693,0.003985,1.154573
